In [1]:
import pandas as pd
import numpy as np
import os
import gc
import pickle
from sklearn.model_selection import StratifiedKFold

from ubq_utilities import (
    get_time_series_cross_val_splits, FOLD_NAME, 
    N_FOLD, MIN_TIME_TO_USE, EMBARGO, TIME_COL,
    STARTING_FEATURE_NAME, TARGET_COL, SUBSAMPLE, STARTING_NUMERIC_FEAT_NAME,
    RANDOM_STATE, SUBSAMPLE_FOLD,
)

TOP_N=100
path_data = '../input/ubq-import'

In [2]:
train = pd.read_parquet(os.path.join(path_data, 'train.parquet'))
gc.collect()

30

In [3]:
gc.collect()

21

In [4]:
print(f"Number of step per fold: {(train['time_id'].nunique()-MIN_TIME_TO_USE)//N_FOLD}")
print(f"Number of step cleared with embargo: {EMBARGO}")

Number of step per fold: 370
Number of step cleared with embargo: 50


In [5]:
gc.collect()

42

In [6]:
train_16 = train.copy()
train_16[STARTING_NUMERIC_FEAT_NAME+[TARGET_COL]] = train_16[STARTING_NUMERIC_FEAT_NAME+[TARGET_COL]].to_numpy(dtype='float16')
gc.collect()

0

In [7]:
del train
gc.collect()

21

In [8]:
print('Saving float16')
train_16.to_pickle('train_16.pkl')

Saving float16


In [9]:
all_feature_corrs_split = train_16.groupby(TIME_COL).apply(
        lambda x: x[STARTING_NUMERIC_FEAT_NAME].corrwith(x[TARGET_COL])
)

#find best corr sharp feature with target on time id
sorted_corr_sharp_split = all_feature_corrs_split.mean().abs()
sorted_corr_sharp_split.sort_values(ascending = False, inplace = True)
gc.collect()

0

In [10]:
best_feature=sorted_corr_sharp_split.index[:TOP_N]
gc.collect()

21

In [11]:
other_features=[]
for i, col in enumerate(best_feature):
    print(i, end="\r")
    mapper_mean = train_16.groupby(['time_id'])[col].mean().to_dict()
    train_16[f'time_id_{col}_mean'] = train_16['time_id'].map(mapper_mean)
    train_16[f'time_id_{col}_mean'] = train_16[f'time_id_{col}_mean'].astype(np.float16)
    other_features.append(f'time_id_{col}_mean')
        
gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


0

In [12]:
train_16['all_mean'] = train_16[STARTING_NUMERIC_FEAT_NAME].mean(axis=1)
train_16['all_std'] = train_16[STARTING_NUMERIC_FEAT_NAME].std(axis=1)
train_16['all_quantile_10'] = train_16[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.1, axis=1)
train_16['all_quantile_50'] = train_16[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.5, axis=1)
train_16['all_quantile_90'] = train_16[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.9, axis=1)

other_features += ['all_mean', 'all_std', 'all_quantile_10', 'all_quantile_50', 'all_quantile_90']

In [13]:
print('Saving float16 with FE')
train_16.to_pickle('train_16_fe.pkl')

Saving float16 with FE


In [14]:
ALL_FEATURE = STARTING_NUMERIC_FEAT_NAME + other_features

with open('ALL_FEATURE.pkl', 'wb') as file:
    pickle.dump(ALL_FEATURE, file)
    
with open('other_features.pkl', 'wb') as file:
    pickle.dump(other_features, file)